# JETANK 색상 인식

이 문서는 frame 내의 특정 pixel들의 HSV 값을 얻기 위해 사용되며, 이는 후속 컬러 추적 기능에 사용될 수 있다. 

HSV 컬러 wheel은 원뿔이나 원기둥처럼 보이지만, 하지만 항상 다음 세 가지 구성 요소가 있다: 모델의 색상 부분은 Hue이고, 0~360도(OpenCV에서는 0~180도)의 숫자로 표시된다: 빨간색은 0~60도 사이이다. 노란색은 61~120도 사이이다. 녹색은 121~180도 사이이다. 시안은 181~240도 사이이다.

RGB 색상 값은 다음과 같이 지정된다: rgb(red, green, blue). 각 매개 변수(적색, 녹색, 청색)는 색상의 강도를 0에서 255 사이의 정수로 정의한다. 예를 들어 rgb(0, 0, 255)가 파란색으로 렌더링되는 것은, 파란색 매개 변수가 가장 높은 값(255)으로 설정되고 다른 매개 변수가 0으로 설정되기 때문이다.

따라서 색상 인식 기능에서는 환경적인 화이트 밸런스와 빛의 밝기 차이의 영향을 피하기 위해, HSV 색상 공간을 사용하여 색상을 일치시킨다.


# 카메라 기능 라이브러리 가져오기

다음 코드 블록을 실행한 후, 잠시 기다렸다가 카메라가 초기화될 때까지 기다린다. 초기화가 성공한 후 코드 블록 아래에 300x300 크기의 실시간 비디오 화면이 나타난다.

이 화면을 마우스 오른쪽 단추로 클릭한 후 `Create New View for Output`를 클릭하면, 다시 창에 카메라 화면을 넣을 수 있다. 문서의 다른 부분을 탐색하더라도, 언제든지 카메라 화면을 볼 수 있다. 이 방법은 다른 위젯에도 적용된다.

이 코드 블록을 여러 번 실행하면 초기화에 실패할 수 있다. 이 솔루션은 이미 `jetbot.Camera`에 포함되어 있고, 커널을 재시작하기만 하면 되지만, 탭 위의 원 화살표를 사용하지 않도록 주의해라. 그래도 초기화에 실패할 수 있다.

커널을 재시작하는 것은 권장되는 방법이다:
`File Browser` 왼쪽에, 녹색 점이 앞에 있는 (녹색 점은 커널이 작동 중임을 나타낸다) `*.ipynb` 파일을 마우스 오른쪽 버튼으로 클릭하고, `Shut Down Kernel`을 선택하면, 녹색 점이 사라진 다음, 이 탭을 닫고 방금 닫았던 `*.ipynb` 파일을 더블 클릭하여 커널을 재시작합니다.

다음 코드를 다시 실행하면, 카메라가 정상적으로 초기화됩니다.

In [ ]:
import traitlets
import ipywidgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

# 카메라 인스턴스 생성, 너비와 높이는 300x300으로 설정
camera = Camera.instance(width=300, height=300)

image_widget = ipywidgets.Image()  # 이 이미지 위젯의 너비와 높이는 카메라와 일치하지 않아도 됩니다.

# 카메라의 'value' 속성과 이미지 위젯의 'value' 속성을 연결하고, bgr8_to_jpeg 함수를 통해 변환
camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

# 이미지 위젯을 표시
display(image_widget)

# 프레임을 HSV 색 공간으로 변환합니다

다음 코드는 프레임을 HSV 색 공간으로 변환하고, 샘플링이 필요한 픽셀의 정렬을 용이하게 하기 위해 프레임에 일부 요소를 그리기 위해 사용됩니다.

다음 코드를 실행해도 프레임이 변경되지 않습니다. image_widget.value 값이 변경되고 camera.observe() 함수가 호출되어야 최종 효과를 관찰할 수 있습니다. 전자의 값은 처리된 프레임을 표시하는 데 사용되고, 후자의 함수는 새 프레임이 수집된 직후 이미지 처리 관련 메서드를 호출하는 데 사용됩니다.

In [2]:
import cv2

# 이 함수에서 수집된 프레임을 가져와 처리합니다.
def colorSelect(imageInput):
    # 가져온 프레임을 HSV 색 공간으로 변환합니다.
    hsv = cv2.cvtColor(imageInput, cv2.COLOR_BGR2HSV)
    
    # 노이즈에 의한 간섭을 줄이기 위해 흐림 효과를 적용합니다.
    hsv = cv2.blur(hsv, (15, 15))
    
    # 화면 크기가 300x300이므로 중간 픽셀의 위치는 (150, 150)입니다. 중간 픽셀의 HSV 파라미터를 가져옵니다.
    hsvValue = hsv[150, 150]
    
    # 샘플링할 대상을 목표로 하는 중앙에 20x20 크기의 사각형을 그립니다.
    cv2.rectangle(imageInput, (140, 140), (160, 160), (255, 255, 255), 1)
    
    # 프레임에 HSV 파라미터를 표시합니다.
    cv2.putText(imageInput, "HSV:{}".format(hsvValue), (80, 180), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
    
    # 프레임을 반환합니다.
    return imageInput

# 비디오 프레임 및 디스플레이 처리

다음 코드를 실행하면 프레임의 색상이 변경되어 'colorSelect()' 기능에 의해 비디오 프레임이 성공적으로 처리되었음을 알 수 있습니다.

In [3]:
def execute(change):
    global image_widget  # 전역 변수 image_widget 사용 선언
    image = change['new']  # 변화된 이미지 데이터를 추출
    image_widget.value = bgr8_to_jpeg(colorSelect(image))  # 추출된 이미지를 처리하여 JPEG 형식으로 변환 후 위젯에 표시
    
execute({'new': camera.value})  # 카메라의 현재 값을 사용하여 execute 함수 호출
camera.unobserve_all()  # 카메라의 모든 관찰을 중지
camera.observe(execute, names='value')  # 카메라의 'value' 속성 변화를 관찰하고 변화가 있을 때마다 execute 함수 실행


비디오 화면에 작은 흰색 사각형이 표시됩니다. 이 작은 사각형을 사용하여 HSV 값을 얻고자 하는 개체를 정렬합니다.
물체를 정렬한 후에는 사각형 아래의 HSV 값이 일정 범위 내에서 변경됩니다. 다음과 같은 세 가지 값의 최대값과 최소값을 기록합니다:
colorUpper = [44, 255, 255]
색상하부 = [24, 100, 100]

## 질문: 여러 번 표본을 추출하여 평균을 내거나 여러 점의 평균을 취하는 대신 이 값이 빠르게 변해야 하는 이유는 무엇입니까?
왜냐하면 블러링 후에 이 픽셀은 실제로 주변 픽셀들과 평균을 낸 결과이기 때문입니다. 이 영역에서 개별 픽셀들의 값 점프 범위를 고려해야 하기 때문에 여러 픽셀들을 모아서 평균을 낼 수 없습니다. 이렇게 하면 실제 화면에서 색상의 HSV 값 범위가 크게 줄어들고 이후 색상 인식 관련 기능들의 작업 효율이 영향을 받게 됩니다.

# 이 기능 처리를 끄고 카메라를 중지합니다
다음 코드 블록을 실행하여 이미지 처리 기능을 끕니다.

In [4]:
camera.unobserve(execute, names='value')  # 카메라의 'value' 속성에 대한 관찰을 중지하여 execute 함수 호출을 멈춤

다시 말씀드리지만, 나중에 노트북에 있는 카메라를 사용할 수 있도록 카메라 커넥터를 제대로 닫도록 하겠습니다.

In [4]:
# camera 정지
camera.stop()